In [7]:
import torch
import torchvision.models as models
import torchvision.ops as ops
from torch import nn
from torchmetrics.classification import MulticlassAccuracy
import math

no_epochs = 50
batch_size = 128

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

acc_function = MulticlassAccuracy(num_classes=102, average='micro').to(device)
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.0001



class DeformConv2d(nn.Module):
    def init(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=False):
        super(DeformConv2d, self).init()
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups

        # Offset convolution
        self.offset_conv = nn.Conv2d(in_channels, 
                                     2 * kernel_size * kernel_size, 
                                     kernel_size=kernel_size, 
                                     stride=stride, 
                                     padding=padding, 
                                     dilation=dilation)

        # Deformable convolution
        self.deform_conv = ops.DeformConv2d(in_channels, 
                                            out_channels, 
                                            kernel_size=kernel_size, 
                                            stride=stride, 
                                            padding=padding, 
                                            dilation=dilation, 
                                            groups=groups,
                                            bias=bias)

        self.init_offset()

    def init_offset(self):
        torch.nn.init.constant_(self.offset_conv.weight, 0)
        torch.nn.init.constant_(self.offset_conv.bias, 0)

    def forward(self, x):
        offset = self.offset_conv(x)
        return self.deform_conv(x, offset)

def replace_conv_with_deform_conv(resnet_layer):
    for i, layer in enumerate(resnet_layer):
        old_conv = getattr(layer, 'conv1')
        setattr(layer, 'conv1', DeformConv2d(old_conv.in_channels, 
                                             old_conv.out_channels, 
                                             kernel_size=old_conv.kernel_size[0], 
                                             stride=old_conv.stride[0], 
                                             padding=old_conv.padding[0], 
                                             bias=False))


def find_learning_rate(model, dataloader, init_value=1e-6, final_value=10., beta=0.98):
    num = len(dataloader) - 1
    mult = (final_value / init_value) ** (1/num)
    lr = init_value
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    avg_loss = 0.
    best_loss = 0.
    batch_num = 0
    losses = []
    log_lrs = []
    for data in dataloader:
        batch_num += 1
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) *loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        # Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 4 * best_loss:
            return log_lrs, losses
        # Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
        # Store the values
        losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        # Do the SGD step
        loss.backward()
        optimizer.step()
        # Update the lr for the next step and store
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
    return log_lrs, losses
        
# Load the ResNet18 model
resnet18 = models.resnet34()#weights=models.ResNet18_Weights.DEFAULT)

# Replace the first convolutional layer in each block with a deformable convolutional layer
# replace_conv_with_deform_conv(resnet18.layer1)
# replace_conv_with_deform_conv(resnet18.layer2)
# replace_conv_with_deform_conv(resnet18.layer3)
replace_conv_with_deform_conv(resnet18.layer4)

# Here, you would define your loss function, optimizer, and other training details
# For the sake of the example, we'll assume those are already defined
# loss_function = ...
# optimizer = ...


# Replace the forward pass to accommodate the custom layers
def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.fc(x)

    return x

resnet18.forward = forward.get(resnet18, models.ResNet)

resnet18.fc = nn.Linear(512,102)

resnet18 = resnet18.to(device)

#lr, _ = find_learning_rate(resnet18, train_data_loader)

ResNet_train_acc, ResNet_rain_loss, ResNet_eval_acc, ResNet_eval_loss, ResNet_test_acc, ResNet_test_loss = train_eval_test(
    resnet18, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

# Now you can proceed to train your modified ResNet18 as usual

AttributeError: 'function' object has no attribute 'DeformConv2d'